In [13]:
# Import dataset
from river import datasets
from pprint import pprint

dataset = datasets.Phishing()
dataset

Phishing websites.

This dataset contains features from web pages that are classified as phishing or not.

    Name  Phishing                                                                           
    Task  Binary classification                                                              
 Samples  1,250                                                                              
Features  9                                                                                  
  Sparse  False                                                                              
    Path  /Users/alp/opt/anaconda3/lib/python3.9/site-packages/river/datasets/phishing.csv.gz

In [14]:
# First data sample
for x, y in dataset:
    pass

x, y = next(iter(dataset))
pprint(x)
pprint(y)

{'age_of_domain': 1,
 'anchor_from_other_domain': 0.0,
 'empty_server_form_handler': 0.0,
 'https': 0.0,
 'ip_in_url': 1,
 'is_popular': 0.5,
 'long_url': 1.0,
 'popup_window': 0.0,
 'request_from_other_domain': 0.0}
True


In [15]:
# Define ML model
from river import linear_model

model = linear_model.LogisticRegression()
model.predict_proba_one(x)

{False: 0.5, True: 0.5}

In [16]:
# Learn from one sample
model = model.learn_one(x,y)
model.predict_proba_one(x)

{False: 0.494687699901455, True: 0.505312300098545}

In [17]:
model.predict_one(x)

True

In [18]:
# Training
from river import metrics

model = linear_model.LogisticRegression()

metric = metrics.ROCAUC()

for x, y in dataset:
    y_pred = model.predict_proba_one(x)
    model.learn_one(x, y)
    metric.update(y, y_pred)
    
metric

ROCAUC: 89.36%

In [19]:
# Evaluate the model
from river import evaluate

model = linear_model.LogisticRegression()
metric = metrics.ROCAUC()

evaluate.progressive_val_score(dataset, model, metric)

ROCAUC: 89.36%

In [20]:
# A common way to improve the performance of a logistic regression is to scale the data
from river import compose
from river import preprocessing

model = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LogisticRegression()
)
model # Visualize the pipeline

Pipeline (
  StandardScaler (
    with_std=True
  ),
  LogisticRegression (
    optimizer=SGD (
      lr=Constant (
        learning_rate=0.01
      )
    )
    loss=Log (
      weight_pos=1.
      weight_neg=1.
    )
    l2=0.
    l1=0.
    intercept_init=0.
    intercept_lr=Constant (
      learning_rate=0.01
    )
    clip_gradient=1e+12
    initializer=Zeros ()
  )
)

In [21]:
# Result after scaling the data
metric = metrics.ROCAUC()
evaluate.progressive_val_score(dataset, model, metric)

ROCAUC: 95.04%